# Demonstration: refine_structure_static

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2016-06-30

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm)

- - -

## Software

This notebook was tested with:

- LAMMPS (Version 2016-03-11)

- Python (Version 2.7.11)

- Jupyter (Version 1.0.0)

__NOTE__: The potential embedded in this Notebook is a MEAM potential.  Running this script with the embedded potential requires that the LAMMPS executable being used has been built with the optional meam package. 

- - -

## Introduction

This Notebook outlines the code performed by the calc_refine_structure_static.py calculation and offers a fully self-contained demonstration of the calculation in action.

The calculation script refines the lattice parameters of an orthogonal system (crystal structure) by calculating the elastic constant using small displacements, and then using the elastic stiffnesses to iterate towards zero pressure. In refining the lattice parameter values, the box dimensions are allowed to relax, but the relative positions of the atoms within the box are held fixed. This provides a quick tool for obtaining both the lattice and elastic constants for a given structure.

__NOTE__: With this method there is no guarantee that the resulting parameters are for a stable structure. Allowing internal relaxations may result in different values for some structures. Additionally, some transformation paths may be restricted from ocurring, which allows for unstable structures to be refined.  

- - -

## 1. Initial Setup

1. __Necessary Python Libraries__: This is a list of the Python libraries used to run this code. 

2. __Set Working Directory__: Change the directory for performing the calculation.

3. __Input Parameter File__: An embedded copy of the input file read by the script is described.

### 1.1 Necessary Python Libraries

This is a list of the Python libraries used to run this code. 

- [numpy](http://www.numpy.org/) (Version 1.10.04)

- [atomman](https://github.com/usnistgov/atomman) (Version 0.6)

- [DataModelDict](https://github.com/usnistgov/DataModelDict) (Version 0.8)

In [1]:
#Standard Python Libraries
import os
import sys
import random
import uuid
from copy import deepcopy

#Additional Python Libraries
import iprPy
from DataModelDict import DataModelDict as DM
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc
import matplotlib.pyplot as plt
import numpy as np

## 1.2 Set Working Directory

This allows for the working directory of the calculation to be changed.

In [15]:
#Specify working directory
working_dir = 'refine_structure_static'


#Check if a relative working_dir matches the current directory's name.
#This ensures that recursive directories are not created if this cell is called multiple times.
cwd = os.path.basename(os.getcwd())
if cwd != working_dir:

    #Move to working_dir and create if needed
    working_dir = os.path.abspath(working_dir)
    try:
        os.chdir(working_dir)
    except:
        os.makedirs(working_dir)
        os.chdir(working_dir)

## 1.2 Input Parameter File

This code creates a copy of the input file as read in by the calculation script.  The input command keywords for this calculation are:

 - __lammps_command__: the path to the executable for running LAMMPS on your system.
 - __mpi_command__: the command associated with calling LAMMPS to run in parallel on your system (optional).
 - __potential_file__: the path to the LAMMPS-potential data model used by atomman to generate the proper LAMMPS commands for an interatomic potential.
 - __potential_dir__: the path to the directory containing any potential artifacts (eg. eam setfl files) that are used. Default is '' (working directory for the simulation.)
 - __load__: the style and path to the initial configuration file being read in. The style can be any file type supported by atomman.System.load()
 - __load_options__: a list of key-value pairs for the optional arguments in atomman.System.load()
 - __symbols__: a space-delimited list of elemental symbols corresponding to the atom types and potential.  Is optional if the elemental information is contained in the file indicated in load.
 - __box_parameters__: optional new box parameters (either a b c, or a b c alpha beta gamma) to scale the loaded system to.
 - __x-axis, y-axis, z-axis__: optional transformation axes for rotating the system. 
 - __atom_shift__: optional positional shift that is applied to all atoms.  Values are in fractional box vectors of the rotated system before size_mults are applied.
 - __size_mults__: optional parameters for supersizing the rotated system by.
 - __length_unit__: defines the unit of length for results, and input parameters if not specified. Default is 'angstrom'.
 - __energy_unit__: defines the unit of energy for results, and input parameters if not specified. Default is 'eV'.
 - __pressure_unit__: defines the unit of pressure for results, and input parameters if not specified. Default is 'GPa'.
 - __strain_range__: specifies the strain range to use when computing the elastic constants. Default is taken to be 1e-5.
 - __pressure_xx, pressure_yy, pressure_zz__: pressure values to equilibriate to. Default values are 0.
 
__NOTE__: The potential_file '2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json' and its associated artifacts are embedded in this Notebook in Sections 2.1 and 2.2, respectively.  The load system_model file 'A1--Cu--fcc.json' is embedded in this Notebook in Section 2.3.  These embedded files are automatically generated when this Notebook is executed allowing for 

In [3]:
with open('calc_refine_structure_static.in', 'w') as input_file:
    input_file.write("""
#Run script for calc_refine_structure_static.py

#Command lines for LAMMPS (and MPI).
#For lammps_command, exclude passing in a script, i.e. no "-in term" or "< term"
lammps_command              lmp_serial
mpi_command                 

#Paths to the potential data model file, and directory containing potential parameters
potential_file              2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json
potential_dir               

#Initial system configuration to load
load                        system_model A1--Cu--fcc.json
load_options                
symbols                     AlS
box_parameters              4.0 4.0 4.0

#System manipulations
x-axis                      
y-axis                      
z-axis                      
atom_shift                  
size_mults                  3 3 3

#Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 

#Run parameters
strain_range                
pressure_xx                 
pressure_yy                 
pressure_zz                 
""")

## 2. Additional Data Files

To make this notebook self-contained, we have embedded a few files.

1. __MEAM Potential.__ Generates library and parameter files for one MEAM potential.

2. __Potential Data Model.__ Collects all input parameters assoiciated with running the MEAM potential in LAMMPS.

3. __Prototype Data Model.__ Generates an instance of the crystal prototype data model used to generate various structures.

### 2.1 MEAM Potential

This generates the library and parameter files associated with the following interatomic potential taken from the [NIST Interatomic Potential Repository](http://www.ctcms.nist.gov/potentials/):

__2012--Jelinek-B--Al-Si-Mg-Cu-Fe__

__Aluminum, Silicon, Magnesium, Copper, and Iron (Al, Si, Mg, Cu, and Fe) Alloys__

*B. Jelinek, S. Groh, M. Horstemeyer, J. Houze, S.G. Kim, G.J. Wagner, A. Moitra, and M.I. Baskes, "Modified embedded atom method potential for Al, Si, Mg, Cu, and Fe alloys," Phys. Rev. B 85, 245102 (2012).*

__Notes__: This file was provided by Bohumir Jelinek (Mississippi State University) and posted on 3 July 2012. He noted, "This is a MEAM potential for Al, Si, Mg, Cu, Fe alloys. It works with LAMMPS, version 19 Jul 2011 or later, when compiled with MEAM support."

__Notice__: Users should consider all available potentials and select one which is appropriate for their use case. Use of this potential within this demonstration should not be construed as an endorsement or a recommendation. 

In [4]:
#Create MEAM library and parameter files for the 2012--Jelinek-B--Al-Si-Mg-Cu-Fe potential
   
#Create MEAM library file "Jelinek_2012_meamf"
with open('Jelinek_2012_meamf', 'w') as f:
    f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meam.alsimgcufe" file providing combinational parameters
#  elt        lat     z       ielement     atwt
#  alpha      b0      b1      b2           b3    alat    esub    asub
#  t0         t1              t2           t3            rozero  ibar

'AlS'        'fcc'   12.     13           26.9815
4.64        2.04    3.00    6.0          1.50  4.05    3.353    1.07
1.0         +4.50           -2.30        8.01          1.0    -5
'SiS'        'dia'   4.      14           28.086
4.87        4.4     5.5     5.5          5.5   5.431   4.63    1.
1.0         2.05            4.47         -1.80         2.2    -5
'MgS'       'hcp'   12.      12           24.305
5.52        4.0    3.0     0.2          1.2  3.194 1.51     0.80
1.0         10.04           9.49         -4.3          0.63   -5
'CuS'        'fcc'   12.     29           63.54
5.11        3.634   2.20    6            2.20  3.62    3.54    1.07
1.0         4.91            2.49         2.95          1.10   -5
'FeS'        'bcc'   8       26           55.847
5.0270      3.500   2       1.00         1     2.851   4.28    0.5550
1          -1.6             12.5          -1.40        1.0    -5""")

#Create MEAM parameter file "Jelinek_2012_meam.alsimgcufe"
with open('Jelinek_2012_meam.alsimgcufe', 'w') as f:
    f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meamf" file providing single element parameters

  Cmin(1,1,1) = 0.8
  repuls(1,1) = 0.1
  Cmin(3,3,3) = 0.8
  Cmin(4,4,4) = 0.8
  Cmin(5,5,5) = 0.68
  repuls(5,5) = 0.3
  Cmax(5,5,5) = 1.9

  nn2(1,1)=1
  nn2(1,2)=1
  nn2(1,3)=1
  nn2(1,4)=1
  nn2(1,5)=1
  nn2(2,2)=1
  nn2(2,3)=1
  nn2(2,4)=1
  nn2(2,5)=1
  nn2(3,3)=1
  nn2(3,4)=1
  nn2(3,5)=1
  nn2(4,4)=1
  nn2(4,5)=1
  nn2(5,5)=1

  lattce(1,2)='b1'
  delta(1,2)=+0.28
  alpha(1,2)=4.56
  re(1,2)=2.62
  Cmin(1,1,2) = 0.50
  Cmin(2,2,1) = 2.00
  Cmin(1,2,1) = 2.00
  Cmin(1,2,2) = 2.00

  lattce(1,3)='b1'
  delta(1,3)=+0.23
  alpha(1,3)=4.52
  re(1,3)=2.87
  Cmin(1,1,3) = 2.00
  Cmin(3,3,1) = 0.00
  Cmin(1,3,1) = 2.00
  Cmin(1,3,3) = 0.00

  lattce(1,4)='b1'
  delta(1,4)=+0.19
  alpha(1,4)=4.65
  re(1,4)=2.53
  Cmin(1,1,4) = 0.00
  Cmin(4,4,1) = 2.00
  Cmin(1,4,1) = 2.00
  Cmin(1,4,4) = 2.00

  lattce(1,5)='b1'
  delta(1,5)=+0.26
  alpha(1,5)=4.64
  re(1,5)=2.45
  Cmin(1,1,5) = 0.90
  Cmin(5,5,1) = 0.10
  Cmin(1,5,1) = 2.00
  Cmin(1,5,5) = 2.00

  lattce(2,3)='b1'
  delta(2,3)=+0.2
  alpha(2,3)=4.73
  re(2,3)=2.75
  Cmin(2,2,3) = 1.00
  Cmin(3,3,2) = 1.00
  Cmin(2,3,2) = 2.00
  Cmin(2,3,3) = 2.00

  lattce(2,4)='b1'
  delta(2,4)=+0.14
  alpha(2,4)=4.74
  re(2,4)=2.46
  Cmin(2,2,4) = 0.00
  Cmin(4,4,2) = 0.00
  Cmin(2,4,2) = 2.00
  Cmin(2,4,4) = 2.00

  lattce(2,5)='b1'
  delta(2,5)=-0.07
  alpha(2,5)=5.17
  re(2,5)=2.39
  Cmin(2,2,5) = 1.00
  Cmin(5,5,2) = 1.00
  Cmin(2,5,2) = 2.00
  Cmin(2,5,5) = 0.00
  attrac(2,5) = 0.1
  repuls(2,5) = 0.1

  lattce(3,4)='b1'
  delta(3,4)=+0.23
  alpha(3,4)=4.70
  re(3,4)=2.63
  Cmin(3,3,4) = 2.00
  Cmin(4,4,3) = 0.00
  Cmin(3,4,3) = 2.00
  Cmin(3,4,4) = 2.00

  lattce(3,5)='b1'
  delta(3,5)=+0.6
  alpha(3,5)=4.96
  re(3,5)=2.61
  Cmin(3,3,5) = 0.65
  Cmin(5,5,3) = 0.00
  Cmin(3,5,3) = 2.00
  Cmin(3,5,5) = 2.00

  lattce(4,5)='b1'
  delta(4,5)=+0.63
  alpha(4,5)=5.21
  re(4,5)=2.42
  Cmin(5,5,4)=0.00

  attrac(5,2) = 0.1
  repuls(5,2) = 0.1

  rc = 5.0
  ialloy=1
  augt1=0
  delr=0.25658351
  emb_lin_neg=1
  bkgd_dyn=1""")

### 2.2 Potential Data Model

This generates the interatomic potential data model file associated with the 2012--Jelinek-B--Al-Si-Mg-Cu-Fe potential.

In [5]:
#Creates a interatomic potential data model for using the MEAM potential in LAMMPS
with open('2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json', 'w') as f:
    f.write("""{
    "LAMMPS-potential": {
        "potential": {
            "key": "1515dd80-1984-49de-bc92-d5724059ff56", 
            "id": "2012--Jelinek-B--Al-Si-Mg-Cu-Fe"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": [
            {
                "symbol": "AlS",
                "element": "Al", 
                "mass": 26.9815                
            }, 
            {
                "symbol": "SiS",
                "element": "Si", 
                "mass": 28.086                
            }, 
            {
                "symbol": "MgS", 
                "element": "Mg",
                "mass": 24.305                
            }, 
            {
                "symbol": "CuS", 
                "element": "Cu", 
                "mass": 63.54
            }, 
            {
                "symbol": "FeS", 
                "element": "Fe", 
                "mass": 55.847
            }
        ], 
        "pair_style": {
            "type": "meam"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "Jelinek_2012_meamf"
                }, 
                {
                    "symbolsList": "True"
                }, 
                {
                    "file": "Jelinek_2012_meam.alsimgcufe"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}""")

### 2.3 Prototype Data Model

This generates the crystal prototype data model file associated with the fcc crystal prototype.

In [6]:
#Creates an instance of a crystal prototype data model for generating crystal structures.
with open('A1--Cu--fcc.json', 'w') as f:
    f.write("""{
    "crystal-prototype": {
        "identifier": {
            "common": "face-centered cubic", 
            "tag": "fcc", 
            "prototype": "Cu", 
            "Pearson-symbol": "cF4", 
            "Strukturbericht": "A1"
        }, 
        "space-group": {
            "number": 225, 
            "Hermann-Maguin": "F m -3 m", 
            "Schoenflies": "O^5_h", 
            "Wykoff": {
                "letter": "a", 
                "multiplicity": 4
            }
        }, 
        "atomic-system": {
            "cell": {
                "cubic": {
                    "a": {
                        "value": 1.0, 
                        "unit": "scaled"
                    }
                }
            }, 
            "atom": [
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.0, 
                            0.0, 
                            0.0
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.0, 
                            0.5, 
                            0.5
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.5, 
                            0.0, 
                            0.5
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.5, 
                            0.5, 
                            0.0
                        ], 
                        "unit": "scaled"
                    }
                }
            ]
        }
    }
}""")

# 3. LAMMPS Script Templates and Generation Functions

This section contains template versions of LAMMPS simulation scripts that are used by this calculation and the functions used to fill in the templates.

1. __LAMMPS Templates__: The embedded template file(s) which LAMMPS input scripts are generated from.

2. __Script Generation Functions__: The function(s) that fill in the templates using supplied parameters.

## 3.1 LAMMPS Templates

In [7]:
with open('cij.template', 'w') as f:
    f.write("""#LAMMPS input script that evaluates the Virial pressure of a system both as
#it was given, and with small strains applied.
#This can be used to estimate the 0 K elastic constants of the system.

<atomman_system_info>

<atomman_pair_info>

#Specify strain range and number of points to examine in the range
variable delta equal <delta>
variable steps equal <steps>
variable deltax equal ${delta}/(${steps}-1)

#Specify variables of the initial configuration's dimensions
variable lx0 equal $(lx)
variable ly0 equal $(ly)
variable lz0 equal $(lz)

#Specify the thermo properties to calculate
variable peatom equal pe/atoms
thermo_style custom step lx ly lz yz xz xy pxx pyy pzz pyz pxz pxy v_peatom pe
thermo_modify format float %.13e

#Compute properties for the initial configuration
run 0

#Compute properties for normal x-direction strains
variable aratio equal 1-${delta}/2.+(v_a-1)*${deltax}
variable xmax equal v_aratio*${lx0}
label loopa
variable a loop ${steps}
change_box all x final 0 ${xmax} remap units box
run 0
next a
jump cij.in loopa
change_box all x final 0 ${lx0} remap units box

#Compute properties for normal y-direction strains
variable bratio equal 1-${delta}/2.+(v_b-1)*${deltax}
variable ymax equal v_bratio*${ly0}
label loopb
variable b loop ${steps}
change_box all y final 0 ${ymax} remap units box
run 0
next b
jump cij.in loopb
change_box all y final 0 ${ly0} remap units box

#Compute properties for normal z-direction strains
variable cratio equal 1-${delta}/2.+(v_c-1)*${deltax}
variable zmax equal v_cratio*${lz0}
label loopc
variable c loop ${steps}
change_box all z final 0 ${zmax} remap units box
run 0
next c
jump cij.in loopc
change_box all z final 0 ${lz0} remap units box

change_box all triclinic

#Compute properties for yz shear strains
variable eyz equal (-${delta}/2.+(v_d-1)*${deltax})*${lz0}
label loopd
variable d loop ${steps}
change_box all yz final ${eyz} remap units box
run 0
next d
jump cij.in loopd
change_box all yz final 0 remap units box

#Compute properties for xz shear strains
variable exz equal (-${delta}/2.+(v_e-1)*${deltax})*${lz0}
label loope
variable e loop ${steps}
change_box all xz final ${exz} remap units box
run 0
next e
jump cij.in loope
change_box all xz final 0 remap units box

#Compute properties for xy shear strains
variable exy equal (-${delta}/2.+(v_f-1)*${deltax})*${ly0}
label loopf
variable f loop ${steps}
change_box all xy final ${exy} remap units box
run 0
next f
jump cij.in loopf
change_box all xy final 0 remap units box""")

## 3.2 Script Generation Functions

In [8]:
def cij_script(template_file, system_info, pair_info, delta = 1e-5, steps = 2):
    """Create lammps script that strains a crystal in each direction x,y,z and shear yz,xz,xy independently."""    
    
    with open(template_file) as f:
        template = f.read()
    variable = {'atomman_system_info': system_info,
                'atomman_pair_info':   pair_info,
                'delta': delta, 
                'steps': steps}
    return '\n'.join(iprPy.tools.fill_template(template, variable, '<', '>'))

## 4. Python Calculation Function(s)

In [9]:
def quick_a_Cij(lammps_exe, cell, potential, symbols, p_xx=0.0, p_yy=0.0, p_zz=0.0, tol=1e-10, diverge_scale=3., delta = 1e-5):
    """
    Quickly refines static orthorhombic cell by evaluating the elastic constants and the virial pressure.
    
    Keyword Arguments:
    lammps_exe -- directory location for lammps executable
    system -- atomman.System to statically deform and evaluate a,b,c and Cij at a given pressure
    potential -- atomman.lammps.Potential representation of a LAMMPS implemented potential
    symbols -- list of element-model symbols for the Potential that correspond to the System's atypes
    pxx, pyy, pzz -- tensile pressures to equilibriate to.  Default is 0.0 for all.  
    tol -- the relative tolerance criterion for identifying box size convergence. Default is 1e-10.
    diverge_scale -- identifies a divergent system if x / diverge_scale < x < x * diverge_scale is not True for x = a,b,c.
    """
    
    #initial parameter setup
    converged = False                   #flag for if values have converged
    
    #define boxes for iterating
    cell_current = deepcopy(cell)       #cell with box parameters being evaluated
    cell_old = None                     #cell with previous box parameters evaluated
    
    for cycle in xrange(100):
        
        #Run LAMMPS and evaluate results based on cell_old
        results = calc_cij(lammps_exe, cell_current, potential, symbols, p_xx, p_yy, p_zz, delta)
        cell_new = results['cell_new']
        
        #Test if box has converged to a single size
        if np.allclose(cell_new.box.vects, cell_current.box.vects, rtol=tol):
            converged = True
            break
        
        #Test if box has converged to two sizes
        elif cell_old is not None and np.allclose(cell_new.box.vects, cell_old.box.vects, rtol=tol):
            #Run LAMMPS Cij script using average between alat0 and alat1
            box = am.Box(a = (cell_new.box.a + cell_old.box.a) / 2.,
                         b = (cell_new.box.b + cell_old.box.b) / 2.,
                         c = (cell_new.box.c + cell_old.box.c) / 2.)
            cell_current.box_set(vects=box.vects, scale=True)
            results = calc_cij(lammps_exe, cell_current, potential, symbols, p_xx, p_yy, p_zz, delta)                 
            
            converged = True
            break
        
        #Test if values have diverged from initial guess
        elif cell_new.box.a < cell.box.a / diverge_scale or cell_new.box.a > cell.box.a * diverge_scale:
            raise RuntimeError('Divergence of box dimensions')
        elif cell_new.box.b < cell.box.b / diverge_scale or cell_new.box.b > cell.box.b * diverge_scale:
            raise RuntimeError('Divergence of box dimensions')
        elif cell_new.box.c < cell.box.c / diverge_scale or cell_new.box.c > cell.box.c * diverge_scale:
            raise RuntimeError('Divergence of box dimensions')  
        elif results['ecoh'] == 0.0:
            raise RuntimeError('Divergence: cohesive energy is 0')
                
        #if not converged or diverged, update cell_old and cell_current
        else:
            cell_old, cell_current = cell_current, cell_new
    
    #Return values if converged
    if converged:        
        return results
    else:
        raise RuntimeError('Failed to converge after 100 cycles')

In [10]:
def calc_cij(lammps_exe, cell, potential, symbols, p_xx=0.0, p_yy=0.0, p_zz=0.0, delta=1e-5):
    """Runs cij_script and returns current Cij, stress, Ecoh, and new cell guess."""
    
    #setup system and pair info
    system_info = lmp.sys_gen(units =       potential.units,
                              atom_style =  potential.atom_style,
                              ucell =       cell,
                              size =        np.array([[0,1], [0,1], [0,1]]))

    pair_info = potential.pair_info(symbols)
    
    #create script and run
    with open('cij.in','w') as f:
        f.write(cij_script('cij.template', system_info, pair_info, delta=delta, steps=2))
    data = lmp.run(lammps_exe, 'cij.in')
    
    #get units for pressure and energy used by LAMMPS simulation
    lmp_units = lmp.style.unit(potential.units)
    p_unit = lmp_units['pressure']
    e_unit = lmp_units['energy']
    
    #Extract thermo values. Each term ranges i=0-12 where i=0 is undeformed
    #The remaining values are for -/+ strain pairs in the six unique directions
    lx = np.array(data.finds('Lx'))
    ly = np.array(data.finds('Ly'))
    lz = np.array(data.finds('Lz'))
    xy = np.array(data.finds('Xy'))
    xz = np.array(data.finds('Xz'))
    yz = np.array(data.finds('Yz'))
    
    pxx = uc.set_in_units(np.array(data.finds('Pxx')), p_unit)
    pyy = uc.set_in_units(np.array(data.finds('Pyy')), p_unit)
    pzz = uc.set_in_units(np.array(data.finds('Pzz')), p_unit)
    pxy = uc.set_in_units(np.array(data.finds('Pxy')), p_unit)
    pxz = uc.set_in_units(np.array(data.finds('Pxz')), p_unit)
    pyz = uc.set_in_units(np.array(data.finds('Pyz')), p_unit)
    
    pe = uc.set_in_units(np.array(data.finds('peatom')), e_unit)
    
    #Set the six non-zero strain values
    strains = np.array([ (lx[2] -  lx[1])  / lx[0],
                         (ly[4] -  ly[3])  / ly[0],
                         (lz[6] -  lz[5])  / lz[0],
                         (yz[8] -  yz[7])  / lz[0],
                         (xz[10] - xz[9])  / lz[0],
                         (xy[12] - xy[11]) / ly[0] ])

    #calculate cij using stress changes associated with each non-zero strain
    cij = np.empty((6,6))
    for i in xrange(6):
        delta_stress = np.array([ pxx[2*i+1]-pxx[2*i+2],
                                  pyy[2*i+1]-pyy[2*i+2],
                                  pzz[2*i+1]-pzz[2*i+2],
                                  pyz[2*i+1]-pyz[2*i+2],
                                  pxz[2*i+1]-pxz[2*i+2],
                                  pxy[2*i+1]-pxy[2*i+2] ])

        cij[i] = delta_stress / strains[i] 
        
    for i in xrange(6):
        for j in xrange(i):
            cij[i,j] = cij[j,i] = (cij[i,j] + cij[j,i]) / 2

    C = am.tools.ElasticConstants(Cij=cij)
    
    if np.allclose(C.Cij, 0.0):
        raise RuntimeError('Divergence of elastic constants to <= 0')
    try:
        S = C.Sij
    except:
        raise RuntimeError('singular C:\n'+str(C.Cij))

    
    #extract the current stress state
    stress = -1 * np.array([[pxx[0], pxy[0], pxz[0]],
                            [pxy[0], pyy[0], pyz[0]],
                            [pxz[0], pyz[0], pzz[0]]])
    
    s_xx = stress[0,0] + p_xx
    s_yy = stress[1,1] + p_yy
    s_zz = stress[2,2] + p_zz
    
    new_a = cell.box.a / (S[0,0]*s_xx + S[0,1]*s_yy + S[0,2]*s_zz + 1)
    new_b = cell.box.b / (S[1,0]*s_xx + S[1,1]*s_yy + S[1,2]*s_zz + 1)
    new_c = cell.box.c / (S[2,0]*s_xx + S[2,1]*s_yy + S[2,2]*s_zz + 1)
    
    if new_a <= 0 or new_b <= 0 or new_c <=0:
        raise RuntimeError('Divergence of box dimensions to <= 0')
    
    newbox = am.Box(a=new_a, b=new_b, c=new_c)
    cell_new = deepcopy(cell)
    cell_new.box_set(vects=newbox.vects, scale=True)
    
    return {'C':C, 'stress':stress, 'ecoh':pe[0], 'cell_new':cell_new}

## 5. Run Calculation and Display Results

### 5.1 Run Calculation

In [11]:
#Read in parameters from input file
with open('calc_refine_structure_static.in') as f:
    input_dict = iprPy.calculation_read_input('refine_structure_static', f)

#read in potential
potential = lmp.Potential(input_dict['potential'], input_dict['potential_dir'])        

#Run quick_a_Cij to refine values
results_dict = quick_a_Cij(input_dict['lammps_command'], 
                           input_dict['initial_system'], 
                           potential, 
                           input_dict['symbols'], 
                           p_xx = input_dict['pressure_xx'], 
                           p_yy = input_dict['pressure_yy'], 
                           p_zz = input_dict['pressure_zz'],
                           delta = input_dict['strain_range'])

#Save data model of results 
results = iprPy.calculation_data_model('refine_structure_static', input_dict, results_dict)
with open('results.json', 'w') as f:
    results.json(fp=f, indent=4)

### 5.2 Display results.json

In [12]:
print results.json(indent=4)

{
    "calculation-system-relax": {
        "calculation": {
            "id": "bc3059ef-638a-4315-b5b6-416c368d26be", 
            "script": "calc_refine_structure_static", 
            "run-parameter": {
                "strain-range": 1e-05, 
                "size-multipliers": {
                    "a": [
                        0, 
                        3
                    ], 
                    "b": [
                        0, 
                        3
                    ], 
                    "c": [
                        0, 
                        3
                    ]
                }
            }
        }, 
        "potential": {
            "key": "1515dd80-1984-49de-bc92-d5724059ff56", 
            "id": "2012--Jelinek-B--Al-Si-Mg-Cu-Fe"
        }, 
        "system-info": {
            "artifact": {
                "file": "A1--Cu--fcc.json", 
                "format": "system_model", 
                "family": "A1--Cu--fcc"
            }, 
            "symb

### 5.3 Print Results

In [13]:
relaxed_box = results.find('relaxed-atomic-system')['cell']

crystal_family = relaxed_box.keys()[0]
print crystal_family

if 'a' in relaxed_box[crystal_family]: 
    print 'a = %.3f %s' % (relaxed_box[crystal_family]['a']['value'], relaxed_box[crystal_family]['a']['unit'])
if 'b' in relaxed_box[crystal_family]: 
    print 'b = %.3f %s' % (relaxed_box[crystal_family]['b']['value'], relaxed_box[crystal_family]['b']['unit']) 
if 'c' in relaxed_box[crystal_family]: 
    print 'c = %.3f %s' % (relaxed_box[crystal_family]['c']['value'], relaxed_box[crystal_family]['c']['unit'])

ecoh = results.find('cohesive-energy')
print 'Ecoh = %.4f %s' % (ecoh['value'], ecoh['unit'])

Cij = results.find('elastic-constants')
for C in Cij.aslist('C'):
    ij = ''.join(C['ij'].split())
    print 'C%s = %.3f %s' % (ij, C['stiffness']['value'], C['stiffness']['unit'])

cubic
a = 4.050 angstrom
Ecoh = -3.3530 eV
C11 = 110.529 GPa
C12 = 60.904 GPa
C44 = 28.392 GPa
